<a href="https://colab.research.google.com/github/morenourey/CHAT/blob/main/ML_3_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUP ML 3 - PREDICT

# Librerias

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler


pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")

# Carga modelo

* Cargar el modelo
* Obtener la lista de model features

In [43]:
# Cargar el modelo guardado si existe
with open('best_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [44]:
features = model.feature_names_
features

['rev',
 'mou',
 'totmrc',
 'da',
 'ovrmou',
 'ovrrev',
 'vceovr',
 'datovr',
 'roam',
 'change_mou',
 'change_rev',
 'drop_vce',
 'drop_dat',
 'blck_vce',
 'blck_dat',
 'unan_vce',
 'unan_dat',
 'plcd_vce',
 'plcd_dat',
 'recv_vce',
 'recv_sms',
 'comp_vce',
 'comp_dat',
 'custcare',
 'ccrndmou',
 'cc_mou',
 'inonemin',
 'threeway',
 'mou_cvce',
 'mou_cdat',
 'mou_rvce',
 'owylis_vce',
 'mouowylisv',
 'iwylis_vce',
 'mouiwylisv',
 'peak_vce',
 'peak_dat',
 'mou_peav',
 'mou_pead',
 'opk_vce',
 'opk_dat',
 'mou_opkv',
 'mou_opkd',
 'drop_blk',
 'attempt',
 'complete',
 'callfwdv',
 'callwait',
 'months',
 'uniqsubs',
 'actvsubs',
 'totcalls',
 'totmou',
 'totrev',
 'adjrev',
 'adjmou',
 'adjqty',
 'avgrev',
 'avgmou',
 'avgqty',
 'avg3mou',
 'avg3qty',
 'avg3rev',
 'avg6mou',
 'avg6qty',
 'avg6rev',
 'hnd_price',
 'phones',
 'models',
 'truck',
 'rv',
 'lor',
 'adults',
 'income',
 'numbcars',
 'forgntvl',
 'eqpdays',
 'new_cell_N',
 'new_cell_U',
 'new_cell_Y',
 'crclscod_A',
 'crclsc

# Carga PREDICT dataset

In [45]:
df_predict = pd.read_csv('telecom_churn_PREDICT.csv')

In [46]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 99 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rev               9777 non-null   float64
 1   mou               9777 non-null   float64
 2   totmrc            9777 non-null   float64
 3   da                9777 non-null   float64
 4   ovrmou            9777 non-null   float64
 5   ovrrev            9777 non-null   float64
 6   vceovr            9777 non-null   float64
 7   datovr            9777 non-null   float64
 8   roam              9777 non-null   float64
 9   change_mou        9704 non-null   float64
 10  change_rev        9704 non-null   float64
 11  drop_vce          10000 non-null  float64
 12  drop_dat          10000 non-null  float64
 13  blck_vce          10000 non-null  float64
 14  blck_dat          10000 non-null  float64
 15  unan_vce          10000 non-null  float64
 16  unan_dat          10000 non-null  float64

# ML Preprocessing

Se deben realizar las mismas transformaciones que se aplicaron en el preprocessing del train:
  * Eliminar mismas variables
  * Imputar mismos valores a nulos
  * Aplicar mismo encoding de categoricos
  * ...todo lo necesario para replicar las variables con las que el modelo se entrenó

NO SE DEBEN repetir:
  * Analisis visual de los datos
  * Analisis de correlaciones
  * Analisis de varianza
  * Calculo de metricas que varien con la distribución

## Eliminar variables 

In [47]:
# Detectar filas duplicadas
print('Filas duplicadas:', df_predict.duplicated().sum()) 
print('Filas filas con identificador duplicadas:', df_predict['Customer_ID'].duplicated().sum()) 

Filas duplicadas: 0
Filas filas con identificador duplicadas: 0


In [48]:
df_customer_ids =  df_predict['Customer_ID']

In [49]:
# Eliminar Columnas identificadoras
df_predict = df_predict.drop(columns=['Customer_ID'])

In [50]:
# Eliminar las mismas columnas eliminadas al entrenar el modelo
to_drop = ['asl_flag' , 'refurb_new' , 'hnd_webcap' , 'ownrent' , 'dwlltype' , 'kid0_2' , 'kid3_5' , 'kid6_10' , 'kid11_15' , 'kid16_17' , 'creditcd']

df_predict = df_predict.drop(columns=to_drop)

In [51]:
# Eliminar filas que tienen todos sus valores vacíos:
df_predict = df_predict.dropna(how='all')

In [52]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 87 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rev               9777 non-null   float64
 1   mou               9777 non-null   float64
 2   totmrc            9777 non-null   float64
 3   da                9777 non-null   float64
 4   ovrmou            9777 non-null   float64
 5   ovrrev            9777 non-null   float64
 6   vceovr            9777 non-null   float64
 7   datovr            9777 non-null   float64
 8   roam              9777 non-null   float64
 9   change_mou        9704 non-null   float64
 10  change_rev        9704 non-null   float64
 11  drop_vce          10000 non-null  float64
 12  drop_dat          10000 non-null  float64
 13  blck_vce          10000 non-null  float64
 14  blck_dat          10000 non-null  float64
 15  unan_vce          10000 non-null  float64
 16  unan_dat          10000 non-null  float64

## Imputacion de nulos

In [53]:
df_predict_num = df_predict.select_dtypes(include=['int64', 'float64'])
df_predict_num.isna().sum().sort_values(ascending=False)

numbcars    5798
lor         3688
income      3240
adults      2986
avg6rev     1432
            ... 
mou_peav       0
opk_vce        0
opk_dat        0
mou_opkv       0
eqpdays        0
Length: 77, dtype: int64

### imputar nulos a columnas numericas

In [54]:
def fill_num_na(df_churn_num, columnas_rellenar):
    for col in columnas_rellenar:
        # Proporciones calculadas
        value_counts = df_churn_num[col].value_counts(normalize=True)

        # Lista de valores para rellenar
        fill_values = np.random.choice(value_counts.index, size=df_churn_num[col].isna().sum(), p=value_counts.values)

        # Rellenando los nulos
        df_churn_num.loc[df_churn_num[col].isna(), col] = fill_values

    return df_churn_num

# Columnas a rellenar
columnas_rellenar = ['lor', 'adults', 'income', 'numbcars']

# Rellenando
df_predict = fill_num_na(df_predict, columnas_rellenar)

### imputar nulos a columnas categoricas

In [55]:
cat_columns = df_predict.select_dtypes(include=['object'])

# Rellenar nulos de forma proporcional con los valores de la columna
for col in df_predict:
    if(col not in cat_columns.columns): continue

    if(df_predict[col].value_counts().index.size == 1):
        # En caso de que la columna solo tenga un mismo valor, se rellena con 'None'
        df_predict[col] = df_predict[col].fillna('None')
    else:
        # Proporciones calculadas
        value_counts = df_predict[col].value_counts(normalize=True)

        # Lista de valores para rellenar
        fill_values = np.random.choice(value_counts.index, size=df_predict[col].isna().sum(), p=value_counts.values)

        # Rellenando los nulos
        df_predict.loc[df_predict[col].isna(), col] = fill_values

### dropear al resto

In [56]:
df_predict.dropna(inplace=True)

In [57]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7968 entries, 0 to 9993
Data columns (total 87 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rev               7968 non-null   float64
 1   mou               7968 non-null   float64
 2   totmrc            7968 non-null   float64
 3   da                7968 non-null   float64
 4   ovrmou            7968 non-null   float64
 5   ovrrev            7968 non-null   float64
 6   vceovr            7968 non-null   float64
 7   datovr            7968 non-null   float64
 8   roam              7968 non-null   float64
 9   change_mou        7968 non-null   float64
 10  change_rev        7968 non-null   float64
 11  drop_vce          7968 non-null   float64
 12  drop_dat          7968 non-null   float64
 13  blck_vce          7968 non-null   float64
 14  blck_dat          7968 non-null   float64
 15  unan_vce          7968 non-null   float64
 16  unan_dat          7968 non-null   float64
 17  

In [58]:
df_predict['infobase'].value_counts()

infobase
M    7752
N     216
Name: count, dtype: int64

In [59]:
# en train se transformaron todos los datos de la columna infobase que no fuesen 'M' a 'None'
df_predict['infobase'] = df_predict['infobase'].apply(lambda x: x if x == 'M' else 'None')

In [60]:
df_predict['infobase'].value_counts()

infobase
M       7752
None     216
Name: count, dtype: int64

## reducir la cardinalidad

In [61]:
# Agrupar valores en categoria 'otros' para columnas igual que las obtenidas en el ML_1
values_crclscod = ['A', 'AA', 'B', 'BA', 'C', 'CA', 'DA', 'EA', 'ZA']
values_dwllsize = ['A', 'B', 'C', 'D', 'J', 'K', 'N', 'O']
values_ethnic = ['B', 'F', 'G', 'H', 'I', 'J', 'N', 'O', 'R', 'S', 'U', 'Z']
columnas_agrupar = {'crclscod': values_crclscod,'dwllsize': values_dwllsize, 'ethnic': values_ethnic }

for columna in columnas_agrupar:
  valores_filtrados = columnas_agrupar[columna]

  df_predict[columna] = df_predict[columna].apply(lambda x: x if x in valores_filtrados else 'otros')

## Encodings

# Check model features

* Comprobar que tenemos en el dataset preprocesado todas las model features, de lo contrario no podremos hacer predict.
* Ordenar las variables en mismo orden que las model features

In [62]:
columns_to_encode = ['new_cell' , 'crclscod' , 'prizm_social_one' , 'area' , 'dualband' , 'marital' , 'HHstatin' , 'dwllsize' , 'ethnic' , 'infobase']

encoder = OneHotEncoder(sparse_output=False)
encoded_categorical_data = encoder.fit_transform(df_predict[columns_to_encode])

In [63]:
# Converter os dados codificados para um DataFrame
encoded_columns = encoder.get_feature_names_out(columns_to_encode)
one_hot_encoded_df  = pd.DataFrame(encoded_categorical_data, columns=encoded_columns)

In [64]:
df_predict = pd.concat([df_predict, one_hot_encoded_df], axis=1)

In [65]:
df_predict = df_predict.drop(columns=columns_to_encode)

In [66]:
# drop columnas de alta correlacion

In [67]:
df_predict.columns.to_list()

['rev',
 'mou',
 'totmrc',
 'da',
 'ovrmou',
 'ovrrev',
 'vceovr',
 'datovr',
 'roam',
 'change_mou',
 'change_rev',
 'drop_vce',
 'drop_dat',
 'blck_vce',
 'blck_dat',
 'unan_vce',
 'unan_dat',
 'plcd_vce',
 'plcd_dat',
 'recv_vce',
 'recv_sms',
 'comp_vce',
 'comp_dat',
 'custcare',
 'ccrndmou',
 'cc_mou',
 'inonemin',
 'threeway',
 'mou_cvce',
 'mou_cdat',
 'mou_rvce',
 'owylis_vce',
 'mouowylisv',
 'iwylis_vce',
 'mouiwylisv',
 'peak_vce',
 'peak_dat',
 'mou_peav',
 'mou_pead',
 'opk_vce',
 'opk_dat',
 'mou_opkv',
 'mou_opkd',
 'drop_blk',
 'attempt',
 'complete',
 'callfwdv',
 'callwait',
 'months',
 'uniqsubs',
 'actvsubs',
 'totcalls',
 'totmou',
 'totrev',
 'adjrev',
 'adjmou',
 'adjqty',
 'avgrev',
 'avgmou',
 'avgqty',
 'avg3mou',
 'avg3qty',
 'avg3rev',
 'avg6mou',
 'avg6qty',
 'avg6rev',
 'hnd_price',
 'phones',
 'models',
 'truck',
 'rv',
 'lor',
 'adults',
 'income',
 'numbcars',
 'forgntvl',
 'eqpdays',
 'new_cell_N',
 'new_cell_U',
 'new_cell_Y',
 'crclscod_A',
 'crclsc

# Rescaling

* Si se entrenó el modelo con un dataset estandarizado, estandarizar con mismo scaler.

In [68]:
# MinMaxScaler para las numericas

df_predict.dropna(inplace=True)
df_churn_num = df_predict.select_dtypes(include=['int64', 'float64'])
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_churn_num)
df_churn_num = pd.DataFrame(X_scaled, columns=df_churn_num.columns)
df_churn_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6225 entries, 0 to 6224
Columns: 152 entries, rev to infobase_None
dtypes: float64(152)
memory usage: 7.2 MB


In [69]:
for col in df_churn_num:
    df_predict[col] = df_churn_num[col]

## Reordenar

In [70]:
# Get the columns of df_train
df_train = pd.read_pickle("final_df.pkl")
train_columns = df_train.columns.to_list()
train_columns.remove('churn')

# Reorder df_predict columns to match df_train
df_predict = df_predict[train_columns]

# PREDICT

* predict() y predict_proba()

In [71]:
# leer pickle del modelo
with open('best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

In [72]:
y_pred = best_model.predict(df_predict)
y_pred_prob = best_model.predict_proba(df_predict)

y_pred


array([False, False, False, ...,  True,  True,  True])

In [73]:
y_pred_prob

array([[0.60035025, 0.39964975],
       [0.51044505, 0.48955495],
       [0.73417797, 0.26582203],
       ...,
       [0.29432517, 0.70567483],
       [0.29432517, 0.70567483],
       [0.29432517, 0.70567483]])

In [74]:
# convertir las probabilidades en 1-dimensional id, quedandonos con el mayor
max_prob = np.max(y_pred_prob, axis=1)

In [75]:
max_prob

array([0.60035025, 0.51044505, 0.73417797, ..., 0.70567483, 0.70567483,
       0.70567483])

In [76]:
# Convertir y_pred a una Series de Pandas
y_pred_series = pd.Series(y_pred, name='predicted_class')

# Convertir max_prob a una Series de Pandas
max_prob_series = pd.Series(max_prob, name='max_predicted_prob')

In [77]:
max_prob_series

0       0.600350
1       0.510445
2       0.734178
3       0.570385
4       0.560622
          ...   
6220    0.705675
6221    0.705675
6222    0.705675
6223    0.705675
6224    0.705675
Name: max_predicted_prob, Length: 6225, dtype: float64

# Guarda predicciones

* Guardar las predicciones en data path. Cada fila debe estar etiquetada con el ID.

In [82]:
df_customer_ids

0       1090001
1       1090002
2       1090003
3       1090004
4       1090005
         ...   
9995    1099996
9996    1099997
9997    1099998
9998    1099999
9999    1100000
Name: Customer_ID, Length: 10000, dtype: int64

In [83]:
df_predicted = pd.concat([df_customer_ids,df_predict, y_pred_series, max_prob_series], axis=1)

In [84]:
df_predicted

,Customer_ID,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,change_rev,drop_vce,drop_dat,blck_vce,blck_dat,unan_vce,unan_dat,plcd_vce,plcd_dat,recv_vce,recv_sms,comp_vce,comp_dat,custcare,ccrndmou,cc_mou,inonemin,threeway,mou_cvce,mou_cdat,mou_rvce,owylis_vce,mouowylisv,iwylis_vce,mouiwylisv,peak_vce,peak_dat,mou_peav,mou_pead,opk_vce,opk_dat,mou_opkv,mou_opkd,drop_blk,attempt,complete,callfwdv,callwait,months,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,forgntvl,eqpdays,new_cell_N,new_cell_U,new_cell_Y,crclscod_A,crclscod_AA,crclscod_B,crclscod_BA,crclscod_C,crclscod_CA,crclscod_DA,crclscod_EA,crclscod_ZA,crclscod_otros,prizm_social_one_C,prizm_social_one_R,prizm_social_one_S,prizm_social_one_T,prizm_social_one_U,area_ATLANTIC SOUTH AREA,area_CALIFORNIA NORTH AREA,area_CENTRAL/SOUTH TEXAS AREA,area_CHICAGO AREA,area_DALLAS AREA,area_DC/MARYLAND/VIRGINIA AREA,area_GREAT LAKES AREA,area_HOUSTON AREA,area_LOS ANGELES AREA,area_MIDWEST AREA,area_NEW ENGLAND AREA,area_NEW YORK CITY AREA,area_NORTH FLORIDA AREA,area_NORTHWEST/ROCKY MOUNTAIN AREA,area_OHIO AREA,area_PHILADELPHIA AREA,area_SOUTH FLORIDA AREA,area_SOUTHWEST AREA,area_TENNESSEE AREA,dualband_N,dualband_T,dualband_Y,marital_A,marital_B,marital_M,marital_S,marital_U,infobase_M,infobase_None,HHstatin_A,HHstatin_B,HHstatin_C,HHstatin_G,HHstatin_H,HHstatin_I,dwllsize_A,dwllsize_B,dwllsize_C,dwllsize_D,dwllsize_J,dwllsize_K,dwllsize_N,dwllsize_O,dwllsize_otros,ethnic_B,ethnic_F,ethnic_G,ethnic_H,ethnic_I,ethnic_J,ethnic_N,ethnic_O,ethnic_R,ethnic_S,ethnic_U,ethnic_Z,ethnic_otros,predicted_class,max_predicted_prob
0,1090001,0.036370,0.023356,0.136324,0.006369,0.000780,0.000839,0.000840,0.000000,0.000657,0.375235,0.150756,0.009585,0.000000,0.044223,0.0,0.019704,0.0,0.030899,0.000000,0.002463,0.0,0.029540,0.000000,0.011494,0.004819,0.003609,0.000707,0.007752,0.019594,0.000000,0.002605,0.002972,0.000970,0.001441,0.000494,0.022872,0.0,0.018200,0.0,0.011843,0.000000,0.008439,0.000000,0.045232,0.030899,0.029540,0.0,0.000000,0.000000,0.000000,0.2,0.009498,0.009541,0.017783,0.013509,0.009956,0.009720,0.050312,0.030918,0.029518,0.020013,0.025133,0.039136,0.031232,0.030443,0.041265,0.285714,0.000000,0.0,0.0,0.0,1.000000,0.0,0.250,0.0,1.0,0.120416,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,False,0.600350
1,1090002,0.042282,0.060248,0.110323,0.012739,0.014511,0.015587,0.014624,0.002409,0.012137,0.326532,0.138097,0.043131,0.000000,0.000000,0.0,0.014778,0.0,0.038702,0.001835,0.022906,0.0,0.047458,0.001957,0.031609,0.032530,0.023706,0.019448,0.031008,0.026912,0.000497,0.040304,0.034918,0.017256,0.005764,0.014920,0.028971,0.0,0.035707,0.0,0.037839,0.003247,0.028046,0.000999,0.033007,0.039170,0.048184,0.0,0.009434,0.000000,0.000000,0.2,0.039660,0.039946,0.028528,0.023054,0.041549,0.040415,0.085114,0.129026,0.122726,0.077737,0.087482,0.063428,0.130002,0.125461,0.068776,0.183673,0.090909,0.0,0.0,0.0,1.000000,0.0,0.500,0.5,0.0,0.057471,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.510445
2,1090003,0.035771,0.041247,0.181781,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.358869,0.150656,0.009585,0.000000,0.019971,0.0,0.030542,0.0,0.039950,0.000000,0.019704,0.0,0.043099,0.000000,0.037356,0.022892,0.017977,0.018034,0.038760,0.039198,0.000000,0.013186,0.021545,0.005115,0.085014,0.022701,0.037103,0.0,0.025498,0.0,0.031196,0.

In [85]:
df_predicted.to_pickle("df_predicted.pkl")